In [1]:
import pandas as pd
import requests
import json
import ast
import matplotlib.pyplot as plt
import re
import os
import simplejson
import plotly.express as px
import plotly.graph_objects as go
from oci.signer import Signer


### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [2]:
user_tenancy_id = '' 
user_id = '' 
user_fingerprint = ''
user_private_key_file_location = '



In [3]:
# user_private_key_file_location = 'your_private_key.pem'
auth = Signer(
    tenancy= user_tenancy_id,
    user= user_id,
    fingerprint= user_fingerprint,
    private_key_file_location= user_private_key_file_location,
    pass_phrase= None
)

auth

## Create Project
#### We don't need to create new projects everytime we run this notebook. A project id once created can be used again and again.


In [4]:
url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/projects" #Prod
payload = json.dumps({
  "displayName": "Forecast LA Demo - LiveLabs",
  "compartmentId": '', #tenancy id of environment where you will create project will go here
  "description": "Forecasting Service LA LiveLabs Demo Project",
  "freeformTags": None,
  "definedTags": None,
  "systemTags": None
})
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload, auth=auth)
create_project_response = json.loads(response.text)
project_id = create_project_response['id']

In [8]:
df_primary = pd.read_csv('primary-15-AUTOMOTIVE.csv')
df_add = pd.read_csv('add-15-AUTOMOTIVE.csv')


### Modify date format  to required format "yyyy-mm-dd hh:mm:ss" for inline data


In [9]:
### Do below only if Date not in required format
# df_primary['date'] = pd.to_datetime(df_primary['date'],format='%d/%m/%y').apply(lambda x: str(x))
df_primary.sort_values(by = "date" , inplace = True)          
# df_add['date'] = pd.to_datetime(df_add['date'],format='%d/%m/%y').apply(lambda x: str(x))
df_add.sort_values(by = "date" , inplace = True) 



In [10]:
#primary data
prim_load = df_primary.values.transpose().tolist()

In [11]:
#additional data
add_load = df_add.values.transpose().tolist()

In [12]:
 #tenancy id of environment where you create forecast will go here
 compartment_id = ''

In [13]:
date_col = 'date'
target_col = 'sales'
id_col = 'item_id'
data_frequency = 'DAY'
forecast_frequency = 'DAY'
forecast_horizon  = 14
forecast_name = "Inline Forecasting Service API LA Live Labs "

In [14]:
%%time

url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts"

payload = simplejson.dumps(
{
"displayName": forecast_name,
"compartmentId": compartment_id,
"projectId": project_id,
"forecastCreationDetails": {
    "areCSVResultFilesRequired": True,
    "forecastHorizon": forecast_horizon,
    "confidenceInterval": "90",
    "errorMeasure": "RMSE",
    "forecastTechnique": "ROCV",
    "forecastFrequency": forecast_frequency,
    "isForecastExplanationRequired": True,
    "modelDetails": {
        "models": [
                "SMA",
                "DMA",
                "HWSM",
                "HWSA",
                "SES",
                "DES",
                "PROPHET"
        ]
    },
    "dataSourceDetails": {
        "type": "INLINE",
        "primaryDataSource": {
            "columnData": prim_load,
            "isDataGrouped": True,
            "tsColName": date_col,
            "tsColFormat": "yyyy-MM-dd HH:mm:ss",
            "dataFrequency": data_frequency,
            "columnSchema": [
                    {
                        "columnName": id_col,
                        "dataType": "STRING"
                    },
                    {
                        "columnName": date_col,
                        "dataType": "DATE"
                    },
                    {
                        "columnName": target_col,
                        "dataType": "INT"
                    }
                ]
        },
        "additionalDataSource": {
            "columnData": add_load,
            "isDataGrouped": True,
            "tsColName": date_col,
            "tsColFormat": "yyyy-MM-dd HH:mm:ss",
            "dataFrequency": data_frequency,
            "columnSchema": [
                    {
                        "columnName": id_col,
                        "dataType": "STRING"
                    },
                    {
                        "columnName": "date",
                        "dataType": "DATE"
                    },
                    {
                        "columnName": "onpromotion",
                        "dataType": "INT"
                    }
                ]
        }

    },
    "targetVariables": [
        target_col
    ]
}
}
    , ignore_nan=True
)

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, auth=auth)

CPU times: user 27 ms, sys: 6.18 ms, total: 33.2 ms
Wall time: 4.81 s


In [15]:
create_forecast_response = json.loads(response.text)


In [2]:
create_forecast_id = create_forecast_response['id']
print(create_forecast_id)

In [3]:
create_forecast_id = create_forecast_response['id']
url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts/{}".format(create_forecast_id)

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload, auth=auth)
get_forecast_response = json.loads(response.text)
get_forecast_response

In [4]:
df_forecasts = pd.DataFrame({'forecast_dates':[],'upper':[],'lower':[],'forecast':[], 'series_id':[]})
for i in range(len(get_forecast_response['forecastResult']['forecastSeries'])):
    group = get_forecast_response['forecastResult']['forecastSeries'][i]['targetColumn']
    point_forecast = get_forecast_response['forecastResult']['forecastSeries'][i]['values']
    pred_intervals = pd.DataFrame(get_forecast_response['forecastResult']
                            ['forecastSeries'][i]['predictionInterval'],dtype=float)
    out = pd.DataFrame({"lower" : list(pred_intervals.lower.tolist()[0]),"upper" : list(pred_intervals.upper.tolist()[0])}) #pred_intervals.explode(['lower','upper'])
    out['forecast'] = point_forecast
    out.reset_index(inplace=True, drop=True)
    forecast_dates = pd.DataFrame({'forecast_dates':get_forecast_response['forecastResult']['forecastSeries'][i]['dates']})
    forecasts = pd.concat([forecast_dates,out],axis=1)
    forecasts['series_id'] = group
    df_forecasts = df_forecasts.append(forecasts, ignore_index = False)

file_name = 'forecast_demo.csv'
df_forecasts.to_csv(file_name, index = None)
df_forecasts

In [5]:
get_forecast_response['forecastResult']['metrics']['targetColumns']

In [6]:
url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts/{}/explanations/".format(create_forecast_id)

payload={}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload, auth=auth)
get_forecast_explanations = json.loads(response.text)
get_forecast_explanations

# Global Feature Importance

In [8]:
import plotly.express as px
import plotly.graph_objects as go

def plot_global_feature_importance(get_forecast_explanations):
    df_imps = pd.DataFrame()
    global_feature_importance = get_forecast_explanations['explanations'][0]['globalFeatureImportance']['influencingFeatures']
    df_imps['Feature_Importance'] = global_feature_importance.values()
    df_imps["Feature_Importance"] = df_imps["Feature_Importance"].apply(lambda x:x["normalizedScore"])
    feature_names = global_feature_importance.keys()
    df_imps['Features'] = feature_names

    title = "Global Feature Importance for Timestep "
    fig = px.bar(df_imps, y="Features", x='Feature_Importance', title=title)
    fig.update_traces(marker_color='lightsalmon')
    fig.show()

plot_global_feature_importance(get_forecast_explanations)

In [7]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_local_feature_importance(get_forecast_explanations, time_step):
    df_imps = pd.DataFrame()
    local_feature_importance = get_forecast_explanations['explanations'][0]['localFeatureImportance']['influencingFeatures'][time_step]
    df_imps['Feature_Importance'] = local_feature_importance.values()
    df_imps["Feature_Importance"] = df_imps["Feature_Importance"].apply(lambda x:x["normalizedScore"])
    feature_names = local_feature_importance.keys()
    df_imps['Features'] = feature_names

    title = "Local Feature Importance for Timestep " + str(time_step)
    fig = px.bar(df_imps, y="Features", x='Feature_Importance', title=title)
    fig.update_traces(marker_color='lightsalmon')
    fig.show()

time_step = 2

plot_local_feature_importance(get_forecast_explanations, time_step)